# 整体结构

CNN被用于图像识别、语音识别等各种场合，在图像识别的比赛中，基于深度学习的方法几乎都以CNN为基础。

CNN和之前介绍的神经网络一样，可以像乐高积木一样通过组装层来构建。不过，CNN中新出现了卷积层（Convolution层）和池化层（Pooling层）。这里我们先看一下如何组装层以构建CNN。

之前介绍的神经网络中，相邻层的所有神经元之间都有连接，这称为全连接（fully-connected）。另外，我们用Affine层实现了全连接层。CNN中新增了Convolution层和Pooling层。CNN的层的连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。这可以理解为之前的“Affine - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。

![CNN](img/cnn.avif)

还需要注意的是，在上图的CNN中，靠近输出的层中使用了之前 的“Affine - ReLU”组合。此外，最后的输出层中使用了之前的“Affine - Softmax”组合。这些都是一般的CNN中比较常见的结构。

# 卷积层
## 全连接层存在的问题

之前介绍的全连接的神经网络中使用了全连接层（Affine层）。在全连接层中，相邻层的神经元全部连接在一起，输出的数量可以任意决定。全连接层存在什么问题呢？那就是数据的形状被“忽视”了。全连接层输入时，需要将3维数据拉平为1维数据。图像是3维形状，这个形状中应该含有重要的空间信息。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息。

而卷积层可以保持形状不变。当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层。因此， 在CNN中，可以（有可能）正确理解图像等具有形状的数据。

另外，CNN 中，有时将卷积层的输入输出数据称为特征图（feature map）。其中，卷积层的输入数据称为输入特征图（input feature map），输出数据称为输出特征图（output feature map）。

## 卷积运算

卷积层进行的处理就是卷积运算。卷积运算相当于图像处理中的“滤波器运算”。我们来看一个具体的例子。对于输入数据，卷积运算以一定间隔滑动滤波器的窗口并应用。这里所说的窗口是指上图中灰色的3 × 3的部分。将各个位置上滤波器的元素和输入的对应元素相乘，然后再求和（有时将这个计算称为乘积累加运算）。然后，将这个结果保存到输出的对应位置。将这个过程在所有位置都进行一遍，就可以得到卷积运算的输出。在全连接的神经网络中，除了权重参数，还存在偏置。CNN中，滤波器的参数就对应之前的权重。并且，CNN中也存在偏置。包含偏置的卷积运算的处理流如下图所示。

![bias](img/cnn2.avif)

如上图所示，向应用了滤波器的数据加上了偏置。偏置通常只有1个 （1 × 1）（本例中，相对于应用了滤波器的4个数据，偏置只有1个），这个值会被加到应用了滤波器的所有元素上。

## 填充

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。在下图的例子中，对大小为(4, 4)的输入数据应用了幅度为1的填充。“幅度为1的填充”是指用幅度为1像素的0填充周围。

![Padding](img/padding.avif)

如上图所示，通过填充，大小为(4, 4)的输入数据变成了(6, 6)的形状。 然后，应用大小为(3, 3)的滤波器，生成了大小为(4, 4)的输出数据。这个例子中将填充设成了1，不过填充的值也可以设置成2、3等任意的整数。

使用填充主要是为了调整输出的大小。因为如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为 1，导致无法再应用卷积运算。为了避免出现这样的情况，就要使用填充。在刚才的例子中，将填充的幅度设为 1，那么相对于输入大小(4, 4)，输出大小也保持为原来的(4, 4)。因此，卷积运算就可以在保持空间大小不变的情况下将数据传给下一层。

## 步幅
应用滤波器的位置间隔称为步幅（stride）。之前的例子中步幅都是1，如果将步幅设为2，则如下图所示，应用滤波器的窗口的间隔变为2个元素。

![strade](img/strade.avif)

步幅可以指定应用滤波器的间隔。增大步幅后，输出大小会变小。而增大填充后，输出大小会变大。我们可以将这样的关系写成算式。接下来，我们看一下对于填充和步幅，如何计算输出大小。这里，假设输入大小为(H, W)，滤波器大小为(FH, FW)，输出大小为(OH, OW)，填充为P，步幅为S。此时，输出大小可通过下式进行计算。

$$
\begin{aligned}
OH &=\frac {H+2P-FH} S + 1 \\
OW &= \frac {W+2P-FW} S + 1
\end{aligned}
$$

这里需要注意的是，虽然只要代入值就可以计算输出大小，但是所设定的值必须使两个式子分别可以除尽。当输出大小无法除尽时（结果是小数时），需要采取报错等对策。根据深度学习的框架的不同，当值无法除尽时，有时会向最接近的整数四舍五入，不进行报错而继续运行。

## 3维数据的卷积运算
之前的卷积运算的例子都是以有高、长方向的2维形状为对象的。但是， 图像是3维数据，除了高、长方向之外，还需要处理通道方向。这里，我们按照与之前相同的顺序，看一下对加上了通道方向的3维数据进行卷积运算的例子。

这里以3通道的数据为例， 展示了卷积运算的结果。和2维数据时相比，可以发现纵深方向（通道方向）上特征图增加了。通道方向上有多个特征图时，会按通道进行输入数据和滤波器的卷积运算，并将结果相加，从而得到输出。其运算顺序如下图所示。

![3D CNN](img/cnn_3d.avif)

需要注意的是，在3维数据的卷积运算中，输入数据和滤波器的通道数要设为相同的值。滤波器大小可以设定为任意值（不过，每个通道的滤波器大小要全部相同）。

## 结合方块思考

将数据和滤波器结合长方体的方块来考虑，3维数据的卷积运算会很容易理解。方块是如下图所示的3维长方体。把3维数据表示为多维数组时，书写顺序为（channel, height, width）。比如，通道数为C、高度为H、 长度为W的数据的形状可以写成（C, H, W）。滤波器也一样，要按（channel, height, width）的顺序书写。比如，通道数为C、滤波器高度为FH（Filter Height）、长度为FW（Filter Width）时，可以写成（C, FH, FW）。

在这个例子中，数据输出是1张特征图。所谓1张特征图，换句话说，就是通道数为1的特征图。那么，如果要在通道方向上也拥有多个卷积运算的输出，该怎么做呢？为此，就需要用到多个滤波器（权重）。

通过应用FN个滤波器，输出特征图也生成了FN个。如果将这FN个特征图汇集在一起，就得到了形状为(FN, OH, OW)的方块。将这个方块传给下一层，就是CNN的处理流。关于卷积运算的滤波器，也必须考虑滤波器的数量。因此，作为4维数据，滤波器的权重数据要按(output_channel, input_ channel, height, width)的顺序书写。比如，通道数为3、大小为5 × 5的滤波器有20个时，可以写成(20, 3, 5, 5)。

卷积运算中（和全连接层一样）存在偏置。在上图的例子中，如果进一步追加偏置的加法运算处理，则结果如下图所示。

![3D CNN](img/cnn_3d2.avif)

上图中，每个通道只有一个偏置。这里，偏置的形状是(FN, 1, 1)， 滤波器的输出结果的形状是(FN, OH, OW)。这两个方块相加时，要对滤波器的输出结果(FN, OH, OW)按通道加上相同的偏置值。其可以通过Numpy的广播机制实现。

## 批处理
神经网络的处理中进行了将输入数据打包的批处理。通过批处理，能够实现处理的高效化和学习时对mini-batch的对应。我们希望卷积运算也同样对应批处理。为此，需要将在各层间传递的数据保存为4维数据。具体地讲，就是按(batch_num, channel, height, width) 的顺序保存数据。例如改成对N个数据进行批处理时，其数据形状如下图所示。

![Batch CNN](img/cnn_batch.avif)

上图的批处理版的数据流中，在各个数据的开头添加了批用的维度。像这样，数据作为4维的形状在各层间传递。这里需要注意的是，网络间传递的是4维数据，对这N个数据进行了卷积运算。也就是说，批处理将N次的处理汇总成了1次进行。

# 池化层
池化是缩小高、长方向上的空间的运算。

![pooling](img/pooling.avif)

上图的例子是按步幅2进行2 × 2的Max池化时的处理顺序。“Max 池化”是获取最大值的运算，“2 × 2”表示目标区域的大小。此外，这个例子中将步幅设为了2，一般来说，池化的窗口大小会和步幅设定成相同的值。

除了Max池化之外，还有Average池化等。相对于Max池化是从目标区域中取出最大值，Average池化则是计算目标区域的平均值。在图像识别领域，主要使用Max池化。因此，此处说到“池化层” 时，指的是Max池化。

池化层有以下特征。
- 没有要学习的参数 池化层和卷积层不同，没有要学习的参数。池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。
- 通道数不发生变化 经过池化运算，输入数据和输出数据的通道数不会发生变化。
- 对微小的位置变化具有鲁棒性（健壮）输入数据发生微小偏差时，池化仍会返回相同的结果。因此，池化对输入数据的微小偏差具有鲁棒性。如下图所示，池化会吸收输入数据的偏差（根据数据的不同，结果有可能不一致）。

# 实现卷积层和池化层

## 4维数据
如前所述，CNN中各层间传递的数据是4维数据。所谓4维数据，比如数据的形状是(10, 1, 28, 28)，则它对应10个高为28、长为28、通道为1的数据。如下所示。

In [4]:
import numpy as np
x = np.random.rand(10, 1, 28, 28) # 随机生成数据
print(x.shape)
print(x[0].shape)
print(x[1].shape)

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)


CNN中处理的是4维数据，因此卷积运算的实现看上去会很复杂，但是通过使用下面要介绍的im2col这个技巧，问题就会变得很简单。

## 基于im2col的展开
im2col是一个函数，将输入数据展开以适合滤波器（权重）。如下图所示，对3维的输入数据应用im2col后，数据转换为2维矩阵（正确地讲，是把包含批数量的4维数据转换成了2维数据）。

im2col会把输入数据展开以适合滤波器（权重）。具体地说，如下图所示，对于输入数据，将应用滤波器的区域（3维方块）横向展开为1列。im2col会在所有应用滤波器的地方进行这个展开处理。

![im2col](img/im2col.avif)

在上图中，为了便于观察，将步幅设置得很大，以使滤波器的应用区域不重叠。在实际的卷积运算中，滤波器的应用区域几乎都是重叠的。在滤波器的应用区域重叠的情况下，使用im2col展开后，展开后的元素个数会多于原方块的元素个数。因此，使用im2col的实现存在比普通的实现消耗更 多内存的缺点。但是，汇总成一个大的矩阵进行计算，对计算机的计算颇有益处。

In [1]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    将输入数据展开
    - input_data: 由（数据量，通道，高，长）的4维数组构成的输入数据
    - filter_h: 滤波器的高
    - filter_w: 滤波器的长
    - stride: 步幅
    - pad: 填充
    """
    N, C, H, W = input_data.shape # 获取输入数据的形状
    out_h = (H + 2*pad - filter_h)//stride + 1 # 输出数据的高
    out_w = (W + 2*pad - filter_w)//stride + 1 # 输出数据的长

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # 填充

    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w)) # 生成空数组

    for y in range(filter_h):
        y_max = y + stride*out_h # 滤波器的高的最大值
        for x in range(filter_w):
            x_max = x + stride*out_w # 滤波器的长的最大值
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride] # 从输入数据中取出相应的数据
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # 转换为2维数组

    return col



使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为1列，并计算2个矩阵的乘积即可。这和全连接层的 Affine层进行的处理基本相同。

如下图所示，基于im2col方式的输出结果是2维矩阵。因为CNN中数据会保存为4维数组，所以要将2维输出数据转换为合适的形状。以上就是卷积层的实现流程。

![im2col](img/im2col2.avif)

## 卷积层的实现
现在使用im2col来实现卷积层。这里我们将卷积层实现为名为Convolution的类。

In [2]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W # 卷积核
        self.b = b # 偏置
        self.stride = stride # 步幅
        self.pad = pad # 填充

    def forward(self, x):
        FN, C, FH, FW = self.W.shape # 卷积核的形状
        N, C, H, W = x.shape # 输入数据的形状
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride) # 输出数据的高
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride) # 输出数据的宽

        # 用im2col展开输入数据，并用reshape将滤波器展开为2维数组。然后，计算展开后的矩阵的乘积
        col = im2col(x, FH, FW, self.stride, self.pad) # 将输入数据转换为二维数组
        col_W = self.W.reshape(FN, -1).T # 将卷积核转换为二维数组
        out = np.dot(col, col_W) + self.b # 卷积运算

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # 转换为合适的输出形状
        return out

卷积层的初始化方法将滤波器（权重）、偏置、步幅、填充作为参数接收。滤波器是 (FN, C, FH, FW)的4维形状。这里通过reshape(FN,-1)将参数指定为-1，这是reshape的一个便利的功能。通过在reshape时指定为-1，reshape函数会自动计算-1维度上的元素个数，以使多维数组的元素个数前后一致。forward的实现中，最后会将输出大小转换为合适的形状。转换时使用了NumPy的transpose函数。transpose会更改多维数组的轴的顺序。

以上就是卷积层的forward处理的实现。通过使用im2col进行展开，基本上可以像实现全连接层的Affine层一样来实现。在进行卷积层的反向传播时，必须进行im2col 的逆处理。这可以使用本书提供的col2im函数来进行。除了使用col2im这一点，卷积层的反向传播和Affi ne层的实现方式都一样。

## 池化层的实现

池化层的实现和卷积层相同，都使用im2col展开输入数据。不过，池化的情况下，在通道方向上是独立的，这一点和卷积层不同。

池化层的实现按下面3个阶段进行。
1. 展开输入数据。
2. 求各行的最大值。
3. 转换为合适的输出大小。

各阶段的实现都很简单，只有一两行代码。

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h # 池化区域的高
        self.pool_w = pool_w # 池化区域的长
        self.stride = stride # 步幅
        self.pad = pad # 填充

    def forward(self, x):
        N, C, H, W = x.shape # 获取输入数据的形状
        out_h = int(1 + (H - self.pool_h) / self.stride) # 输出数据的高
        out_w = int(1 + (W - self.pool_w) / self.stride) # 输出数据的长

        # 展开
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad) # 将输入数据转换为二维数组
        col = col.reshape(-1, self.pool_h*self.pool_w) # 转换为合适的形状

        # 最大值
        out = np.max(col, axis=1) # 计算最大值
        # 整形
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2) # 转换为合适的输出形状

        return out

值得注意的是，最大值的计算可以使用 NumPy 的 np.max方法。np.max可以指定axis参数，并在这个参数指定的各个轴方向上求最大值。

以上就是池化层的forward处理的介绍。如上所述，通过将输入数据展开为容易进行池化的形状，后面的实现就会变得非常简单。另外，池化层的backward处理可以参考ReLU层的实现中使用的max的反向传播。

# CNN的实现
我们已经实现了卷积层和池化层，现在来组合这些层，搭建进行手写数字识别的CNN。这里要实现如下图所示的CNN。

![CNN impl](img/cnn_impl.avif)

 如上图所示，网络的构成是“Convolution - ReLU - Pooling -Affine - ReLU - Affine - Softmax”，我们将它实现为名为SimpleConvNet的类。 

In [3]:
import sys, os
sys.path.append(os.pardir)
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        # 卷积层的超参数（字典）
        filter_num = conv_param['filter_num'] # 卷积核数量
        filter_size = conv_param['filter_size'] # 卷积核大小
        filter_pad = conv_param['pad'] # 填充
        filter_stride = conv_param['stride'] # 步幅
        input_size = input_dim[1] # 输入数据的大小（MNIST为28）
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1 # 卷积层输出数据的大小
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2)) # 池化层输出数据的大小

        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size) # 卷积核的权重
        self.params['b1'] = np.zeros(filter_num) # 卷积核的偏置
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size) # 全连接层的权重
        self.params['b2'] = np.zeros(hidden_size) # 全连接层的偏置
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size) # 全连接层的权重
        self.params['b3'] = np.zeros(output_size) # 全连接层的偏置

        # 生成层
        self.layers = OrderedDict() # 有序字典
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad']) # 卷积层
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss() # 输出层

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for key in ['W1', 'b1', 'W2', 'b2', 'W3', 'b3']: # 计算权重参数的梯度
            grads[key] = numerical_gradient(loss_W, self.params[key])

        return grads

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]
    
    def save_params(self, file_name="params.pkl"):
        params = {}

        for key, val in self.params.items():
            params[key] = val

        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)

        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

# CNN的可视化

## 第1层权重的可视化
我们来比较一下学习前和学习后的权重，学习前的滤波器是随机进行初始化的，所以在黑白的浓淡上没有规律可循，但学习后的滤波器变成了有规律的图像。我们发现，通过学习，滤波器被更新成了有规律的滤波器，比如从白到黑渐变的滤波器、含有块状区域（称为blob）的滤波器等。

如果要问上图中右边的有规律的滤波器在“观察”什么，答案就是它在观察边缘（颜色变化的分界线）和斑块（局部的块状区域）等。比如，左半部分为白色、右半部分为黑色的滤波器的情况下，如下图所示，会对垂直方向上的边缘有响应。

由此可知，卷积层的滤波器会提取边缘或斑块等原始信息。而刚才实现的CNN会将这些原始信息传递给后面的层。

## 基于分层结构的信息提取
上面的结果是针对第1层的卷积层得出的。第1层的卷积层中提取了边缘或斑块等“低级”信息，那么在堆叠了多层的CNN中，各层中又会提取什么样的信息呢？根据深度学习的可视化相关的研究，随着层次加深，提取的信息（正确地讲，是反映强烈的神经元）也越来越抽象。

下图中展示了进行一般物体识别（车或狗等）的8层CNN。这个网络结构的名称是下一节要介绍的AlexNet。AlexNet网络结构堆叠了多层卷积层和池化层，最后经过全连接层输出结果。下图的方块表示的是中间数据， 对于这些中间数据，会连续应用卷积运算。

![CNN visualization](img/cnn_visual.webp)

如上图所示，如果堆叠了多层卷积层，则随着层次加深，提取的信息也愈加复杂、抽象，这是深度学习中很有意思的一个地方。最开始的层对简单的边缘有响应，接下来的层对纹理有响应，再后面的层对更加复杂的物体部件有响应。也就是说，随着层次加深，神经元从简单的形状向“高级”信息变化。换句话说，就像我们理解东西的“含义”一样，响应的对象在逐渐变化。

# 代表性的CNN

这里，我们介绍其中特别重要的两个网络，一个是在1998年首次被提出的CNN元祖LeNet， 另一个是在深度学习受到关注的2012年被提出的AlexNet。

## LeNet
LeNet在1998年被提出，是进行手写数字识别的网络。如下图所示，它有连续的卷积层和池化层（正确地讲，是只“抽选元素”的子采样层），最后经全连接层输出结果。

![LeNet](img/lenet.webp)

和“现在的CNN”相比，LeNet有几个不同点。第一个不同点在于激活函数。LeNet中使用sigmoid函数，而现在的CNN中主要使用ReLU函数。此外，原始的LeNet中使用子采样（subsampling）缩小中间数据的大小，而现在的CNN中Max池化是主流。综上，LeNet与现在的CNN虽然有些许不同，但差别并不是那么大。想到LeNet是20多年前提出的最早的CNN，还是很令人称奇的。

## AlexNet
在LeNet问世20多年后，AlexNet被发布出来。AlexNet是引发深度学习热潮的导火线，不过它的网络结构和LeNet基本上没有什么不同，如下图所示。

![AlexNet](img/AlexNet.avif)

AlexNet叠有多个卷积层和池化层，最后经由全连接层输出结果。虽然结构上AlexNet和LeNet没有大的不同，但有以下几点差异。
1. 激活函数使用ReLU。
2. 使用进行局部正规化的LRN（Local Response Normalization）层。
3. 使用Dropout

如上所述，关于网络结构，LeNet和AlexNet没有太大的不同。但是，围绕它们的环境和计算机技术有了很大的进步。具体地说，现在任何人都可以获得大量的数据。而且，擅长大规模并行计算的GPU得到普及，高速进行大量的运算已经成为可能。大数据和GPU已成为深度学习发展的巨大的原动力。